In [35]:
import pyspark
import pandas as pd
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import split, regexp_extract, to_date,when, col

from datetime import datetime
import re

In [36]:
spark = SparkSession.builder.appName('bovo').getOrCreate()
spark

In [37]:

schema = StructType([
    StructField("PredictedArrival", StringType(), False),
    StructField("Flight_ID", StringType(), False),
    StructField("Destination", StringType(), False),
    StructField("Airline", StringType(), False),
    StructField("Aircraft_ID", StringType(), False),
    StructField("NUll", StringType(), False),
    StructField("TimeOfArrival", StringType(), False),
    StructField("Airport", StringType(), False),
    StructField("Date", StringType(), False)
    
    
])

In [38]:
df_pyspark=spark.read.option('header','false').csv(r'C:\Users\httyd\Desktop\capstone\airports\Data\2024-03-13\Arrivals*.csv',schema=schema)

In [39]:
df_pyspark=df_pyspark.na.drop(how="any", thresh=5)
df_pyspark = df_pyspark.drop('NULL')

In [40]:
df_pyspark.show(100)

+----------------+---------+--------------------+--------------------+-------------+----------------+--------------------+--------------------+
|PredictedArrival|Flight_ID|         Destination|             Airline|  Aircraft_ID|   TimeOfArrival|             Airport|                Date|
+----------------+---------+--------------------+--------------------+-------------+----------------+--------------------+--------------------+
|           11:11|   UA6139|      McAllen (MFE)-|     Mesa Airlines -|CRJ9 (N922FJ)|    Landed 10:40|Houston George Bu...|Tuesday, Mar 12 2024|
|           11:12|   UA2385| Grand Rapids (GRR)-|   United Airlines -|A320 (N493UA)|    Landed 10:28|Houston George Bu...|Tuesday, Mar 12 2024|
|           11:13|    AC591|      Toronto (YYZ)-|        Air Canada -|BCS3 (C-GWUS)|    Landed 10:42|Houston George Bu...|Tuesday, Mar 12 2024|
|           11:13|   NK1597|    Las Vegas (LAS)-|   Spirit Airlines -|A20N (N969NK)|    Landed 10:52|Houston George Bu...|Tuesday, Mar 1

In [41]:
df_pyspark = df_pyspark.withColumn("Day", split(df_pyspark["Date"], ",").getItem(0)) \
                       .withColumn("NumericalDate", split(df_pyspark["Date"], ",").getItem(1))
df_pyspark = df_pyspark.drop("Date")
# Extract AirportName and AirportCode using regexp_extract
df_pyspark = df_pyspark.withColumn("AirportName", regexp_extract(df_pyspark["Airport"], r'^(.*?)\(', 1)) \
                       .withColumn("Airport_Code", regexp_extract(df_pyspark["Airport"], r'\((.*?)\)', 1))
df_pyspark = df_pyspark.drop("Airport")
# Extract AirportName and AirportCode using regexp_extract
df_pyspark = df_pyspark.withColumn("DestinationName", regexp_extract(df_pyspark["Destination"], r'^(.*?)\(', 1)) \
                       .withColumn("Destination_Code", regexp_extract(df_pyspark["Destination"], r'\((.*?)\)', 1))
df_pyspark = df_pyspark.drop("Destination")
# Extract AirportName and AirportCode using regexp_extract
df_pyspark = df_pyspark.withColumn("Aircraft_Type", regexp_extract(df_pyspark["Aircraft_ID"], r'^(.*?)\(', 1)) \
                       .withColumn("Aircraft_Code", regexp_extract(df_pyspark["Aircraft_ID"], r'\((.*?)\)', 1))
df_pyspark = df_pyspark.drop("Aircraft_ID")

# Split TimeOfDeparture based on whether it contains "Departed", "Estimated", or "Canceled"
df_pyspark = df_pyspark.withColumn("ArrivalStatus", 
                                   when(col("TimeOfArrival").contains("Landed"), "Landed")
                                   .when(col("TimeOfArrival").contains("Canceled"), "Canceled")
                                   .otherwise("Unknown"))

# Split TimeOfDeparture into two columns based on the DepartureStatus
df_pyspark = df_pyspark.withColumn("ActualArrivalTime", when(col("ArrivalStatus") == "Landed",
                                                               split(col("TimeOfArrival"), " ")[1])
                                   .otherwise(None))

df_pyspark = df_pyspark.drop("TimeOfArrival")



In [42]:
df_pyspark = df_pyspark.dropDuplicates()
row_count = df_pyspark.count()
print("Number of rows in DataFrame:", row_count)

Number of rows in DataFrame: 24882


In [43]:
# Assuming df_pyspark is your DataFrame containing the sample data
df_pyspark = df_pyspark.withColumn("NumericalDate", to_date("NumericalDate", " MMM dd yyyy"))

# Show the transformed DataFrame
df_pyspark.show()

+----------------+---------+--------------------+---------+-------------+--------------------+------------+---------------+----------------+-------------+-------------+-------------+-----------------+
|PredictedArrival|Flight_ID|             Airline|      Day|NumericalDate|         AirportName|Airport_Code|DestinationName|Destination_Code|Aircraft_Type|Aircraft_Code|ArrivalStatus|ActualArrivalTime|
+----------------+---------+--------------------+---------+-------------+--------------------+------------+---------------+----------------+-------------+-------------+-------------+-----------------+
|           05:25|   UA1907|   United Airlines -|Wednesday|   2024-03-13|Houston George Bu...|    IAH/KIAH|Guatemala City |             GUA|        B738 |       N87512|       Landed|            05:24|
|           08:39|   UA2205|   United Airlines -|Wednesday|   2024-03-13|Houston George Bu...|    IAH/KIAH|      Columbus |             CMH|        A319 |       N830UA|       Landed|            08

In [44]:
df_pyspark.show(100)

+----------------+---------+--------------------+---------+-------------+--------------------+------------+----------------+----------------+-------------+-------------+-------------+-----------------+
|PredictedArrival|Flight_ID|             Airline|      Day|NumericalDate|         AirportName|Airport_Code| DestinationName|Destination_Code|Aircraft_Type|Aircraft_Code|ArrivalStatus|ActualArrivalTime|
+----------------+---------+--------------------+---------+-------------+--------------------+------------+----------------+----------------+-------------+-------------+-------------+-----------------+
|           05:25|   UA1907|   United Airlines -|Wednesday|   2024-03-13|Houston George Bu...|    IAH/KIAH| Guatemala City |             GUA|        B738 |       N87512|       Landed|            05:24|
|           08:39|   UA2205|   United Airlines -|Wednesday|   2024-03-13|Houston George Bu...|    IAH/KIAH|       Columbus |             CMH|        A319 |       N830UA|       Landed|         